In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# You may need to change this to get data
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import recall_score, classification_report, accuracy_score, precision_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.utils import to_categorical

import os
import time
import numpy as np
import pandas as pd

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

## Data

In [ ]:
# 1. Specify the folder path containing the CSV files.
folder_path = 'Traning and Testing Tets (5% of the entier dataset)/All features/'

# 2. Initialize an empty list to store DataFrames.
dfs = []

# 3. List CSV files in the folder.
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# 4. Loop through the CSV files, read and append them to the list.
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 5. Concatenate the list of DataFrames into a single DataFrame.
combined_df = pd.concat(dfs, ignore_index=True)

<ipython-input-4-c2ce571afe12>:16: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-4-c2ce571afe12>:16: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-4-c2ce571afe12>:16: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-4-c2ce571afe12>:16: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [ ]:
combined_df.head()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.12704,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.20000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.12704,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.12704,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.12704,96,75,1.135260,1.129970,770,602,1,DoS,HTTP


## Sampling and scaling

In [ ]:
# Load your dataset and prepare X (features) and y (target)
y = combined_df['category']
X = combined_df.drop(['category'], axis=1)
# Undersampling: Reducing the large classes to a chosen number, e.g., 10000 samples
undersample_strategy = {
    'DDoS': 10000,
    'DoS': 10000,
    'Reconnaissance': 10000
}
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy)
X_undersampled, y_undersampled = undersampler.fit_resample(X, y)
# Oversampling: Increasing the very small classes to match the larger ones (e.g., 10000 samples)
oversample_strategy = {
    'Normal': 10000,
    'Theft': 10000
}
oversampler = RandomOverSampler(sampling_strategy=oversample_strategy)
X_resampled, y_resampled = oversampler.fit_resample(X_undersampled, y_undersampled)
# Check the distribution of classes in the resampled dataset
unique_classes_resampled, class_counts_resampled = np.unique(y_resampled, return_counts=True)
print("\nClass distribution in the resampled dataset:")
for cls, count in zip(unique_classes_resampled, class_counts_resampled):
    print(f"Class {cls}: {count} samples")


Class distribution in the resampled dataset:
Class DDoS: 10000 samples
Class DoS: 10000 samples
Class Normal: 10000 samples
Class Reconnaissance: 10000 samples
Class Theft: 10000 samples


In [ ]:
# Convert X_resampled and y_resampled back into a dataframe
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['category'] = y_resampled

resampled_df.head()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,subcategory,category
0,3060163,1.528099e+09,e,1,udp,3,192.168.100.149,41820,192.168.100.3,80,...,0.656362,100,86,0.575563,0.656362,812,602,1,UDP,DDoS
1,2908849,1.528099e+09,e,1,udp,3,192.168.100.149,11084,192.168.100.3,80,...,0.628638,100,88,0.563339,0.628638,896,704,1,UDP,DDoS
2,2219925,1.528096e+09,e s,2,tcp,1,192.168.100.149,10408,192.168.100.3,80,...,0.449254,100,84,0.352210,0.449254,300,300,1,TCP,DDoS
3,3075728,1.528099e+09,e,1,udp,3,192.168.100.148,29807,192.168.100.3,80,...,0.802510,100,100,0.802562,0.802510,1100,1100,1,UDP,DDoS
4,3377280,1.528099e+09,e,1,udp,3,192.168.100.149,11355,192.168.100.3,80,...,0.565491,100,100,0.565491,0.565491,700,700,1,UDP,DDoS


In [ ]:
# Split the features and target variable
X_resampled = resampled_df.drop('category', axis=1)
y_resampled = resampled_df['category']

In [ ]:
# List of chosen features
features = [
    'state_number', 'max', 'mean', 'stddev',
    'min', 'seq', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'TnP_PerProto','dur'
]
# Dropped daddr, state, dport

# Subset the dataframe to only include the desired features
resampled_df_subset = resampled_df[features + ['category']]

# Split the features and target variable
X_resampled_subset = resampled_df_subset.drop('category', axis=1)
y_resampled_subset = resampled_df_subset['category']

# Split into training and testing sets
X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(X_resampled_subset, y_resampled_subset, test_size=0.2, random_state=42)

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit on the training data
X_train_scaled = scaler.fit_transform(X_train_subset)

# Transform the testing data
X_test_scaled = scaler.transform(X_test_subset)

In [ ]:
resampled_df_subset.head()

,state_number,max,mean,stddev,min,seq,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,TnP_PerProto,dur,category
0,4,4.060574,2.703605,1.911742,0.000000,169385,100,86,812,12.162014,DDoS
1,4,4.060251,3.386155,0.950752,2.041589,18071,100,88,896,14.201034,DDoS
2,3,3.628548,2.405469,1.701005,0.000000,44384,100,84,644,14.196070,DDoS
3,4,4.112906,3.656178,0.641040,2.749620,184950,100,100,1100,13.706099,DDoS
4,4,4.148946,2.744936,1.941134,0.000000,224352,100,100,700,12.378614,DDoS


In [ ]:
non_numeric_cols = resampled_df_subset.select_dtypes(exclude=[np.number]).columns
print(non_numeric_cols)

Index(['category'], dtype='object')


## Models

In [ ]:
# Convert string labels to integers
label_encoder = LabelEncoder()
y_train_encoded_int = label_encoder.fit_transform(y_train_subset)
y_test_encoded_int = label_encoder.transform(y_test_subset)

# One-hot encode these integer labels
y_train_encoded = to_categorical(y_train_encoded_int)
y_test_encoded = to_categorical(y_test_encoded_int)

In [ ]:
# Simple NN Model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_train_encoded_int)), activation='softmax'))

# Compile model with accuracy, recall and precision metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])

start_time = time.time()

# Train the model
history = model.fit(X_train_scaled, y_train_encoded, epochs=10, validation_data=(X_test_scaled, y_test_encoded))

elapsed_time = time.time() - start_time

# Evaluate the model
loss, accuracy, recall, precision = model.evaluate(X_test_scaled, y_test_encoded)



print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'Precision: {recall * 100:.2f}%')
print(f"Training Time: {elapsed_time:.2f} seconds")

Epoch 1/10
1250/1250 [==============================] - 6s 3ms/step - loss: 0.4240 - accuracy: 0.8414 - recall_5: 0.7790 - precision_5: 0.8879 - val_loss: 0.2381 - val_accuracy: 0.9074 - val_recall_5: 0.9046 - val_precision_5: 0.9138
Epoch 2/10
1250/1250 [==============================] - 3s 2ms/step - loss: 0.2052 - accuracy: 0.9215 - recall_5: 0.9187 - precision_5: 0.9244 - val_loss: 0.1984 - val_accuracy: 0.9215 - val_recall_5: 0.9176 - val_precision_5: 0.9230
Epoch 3/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.1668 - accuracy: 0.9385 - recall_5: 0.9370 - precision_5: 0.9398 - val_loss: 0.1520 - val_accuracy: 0.9512 - val_recall_5: 0.9506 - val_precision_5: 0.9528
Epoch 4/10
1250/1250 [==============================] - 4s 3ms/step - loss: 0.1433 - accuracy: 0.9475 - recall_5: 0.9464 - precision_5: 0.9486 - val_loss: 0.1322 - val_accuracy: 0.9611 - val_recall_5: 0.9599 - val_precision_5: 0.9616
Epoch 5/10
1250/1250 [==============================] - 3s 2ms/s

In [ ]:
# Reshape input to be [samples, timesteps, features]
X_train_rnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_rnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Create the simple RNN model
model = Sequential()
model.add(SimpleRNN(32, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), activation='relu', return_sequences=True))
model.add(SimpleRNN(32, activation='relu'))
model.add(Dense(len(np.unique(y_train_encoded_int)), activation='softmax'))

# Compile model with accuracy, recall and precision metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])

start_time = time.time()

# Train the model
history = model.fit(X_train_rnn, y_train_encoded, epochs=10, validation_data=(X_test_rnn, y_test_encoded))

elapsed_time = time.time() - start_time

# Evaluate the model
loss, accuracy, recall, precision = model.evaluate(X_test_rnn, y_test_encoded)

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f"Training Time: {elapsed_time:.2f} seconds")

Epoch 1/10
1250/1250 [==============================] - 19s 11ms/step - loss: 0.3184 - accuracy: 0.8756 - recall_3: 0.8504 - precision_3: 0.9021 - val_loss: 0.1939 - val_accuracy: 0.9317 - val_recall_3: 0.9281 - val_precision_3: 0.9336
Epoch 2/10
1250/1250 [==============================] - 7s 5ms/step - loss: 0.1561 - accuracy: 0.9445 - recall_3: 0.9432 - precision_3: 0.9459 - val_loss: 0.1349 - val_accuracy: 0.9496 - val_recall_3: 0.9486 - val_precision_3: 0.9505
Epoch 3/10
1250/1250 [==============================] - 9s 7ms/step - loss: 0.1337 - accuracy: 0.9519 - recall_3: 0.9511 - precision_3: 0.9530 - val_loss: 0.1470 - val_accuracy: 0.9447 - val_recall_3: 0.9441 - val_precision_3: 0.9460
Epoch 4/10
1250/1250 [==============================] - 12s 9ms/step - loss: 0.1276 - accuracy: 0.9532 - recall_3: 0.9525 - precision_3: 0.9542 - val_loss: 0.1149 - val_accuracy: 0.9579 - val_recall_3: 0.9576 - val_precision_3: 0.9585
Epoch 5/10
1250/1250 [==============================] - 15s 1

In [ ]:
print(f'Precision: {precision * 100:.2f}%')

Precision: 95.95%


In [ ]:
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "KNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "SVM": SVC()
}

# Convert string labels to integers for model compatibility
label_encoder = LabelEncoder()
y_train_encoded_int = label_encoder.fit_transform(y_train)
y_test_encoded_int = label_encoder.transform(y_test)

for name, model in models.items():
    start_time = time.time()

    model.fit(X_train_scaled, y_train_encoded_int)

    elapsed_time = time.time() - start_time

    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test_encoded_int, y_pred)
    recall = recall_score(y_test_encoded_int, y_pred, average='macro')
    precision = precision_score(y_test_encoded_int, y_pred, average='macro')

    print(f"Model: {name}")
    print(f"Training Time: {elapsed_time:.2f} seconds")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    #print(classification_report(y_test_encoded_int, y_pred, target_names=label_encoder.classes_))
    print("---------------------------------------------------")

Model: Decision Tree
Training Time: 0.27 seconds
Accuracy: 99.86%
Recall: 99.86%
Precision: 99.86%
---------------------------------------------------
Model: Naive Bayes
Training Time: 0.02 seconds
Accuracy: 81.52%
Recall: 81.54%
Precision: 84.30%
---------------------------------------------------
Model: KNN
Training Time: 0.13 seconds
Accuracy: 98.89%
Recall: 98.90%
Precision: 98.90%
---------------------------------------------------
Model: Logistic Regression
Training Time: 3.84 seconds
Accuracy: 90.55%
Recall: 90.59%
Precision: 90.54%
---------------------------------------------------
Model: Random Forest
Training Time: 5.96 seconds
Accuracy: 99.94%
Recall: 99.94%
Precision: 99.94%
---------------------------------------------------
Model: Gradient Boosting
Training Time: 47.54 seconds
Accuracy: 99.87%
Recall: 99.87%
Precision: 99.87%
---------------------------------------------------
Model: SVM
Training Time: 12.20 seconds
Accuracy: 95.05%
Recall: 95.10%
Precision: 95.24%
-----

In [ ]:
resampled_df_subset["category"].value_counts()

DDoS              10000
DoS               10000
Normal            10000
Reconnaissance    10000
Theft             10000
Name: category, dtype: int64